In [1]:
# difine tokenizer
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")

In [2]:
dataset_max_length = 512

In [3]:
# formatted text
formatted_text = [
 "<|begin_of_text|><|start_header_id|>user<|end_header_id|>Give three tips for staying healthy.<|eot_id|><|start_header_id|>assistant<|end_header_id|>1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. 2. Exercise regularly to keep your body active and strong. 3. Get enough sleep and maintain a consistent sleep schedule.<|eot_id|><|end_of_text|>", 
 "<|begin_of_text|><|start_header_id|>user<|end_header_id|>What are the three primary colors?<|eot_id|><|start_header_id|>assistant<|end_header_id|>The three primary colors are red, blue, and yellow.<|eot_id|><|end_of_text|>"
]

In [4]:
# print formatted text list
input_word_list = []

for i in range(len(formatted_text)):
    input_word = formatted_text[i]
    input_word_list.append(input_word)
    print(input_word_list)

['<|begin_of_text|><|start_header_id|>user<|end_header_id|>Give three tips for staying healthy.<|eot_id|><|start_header_id|>assistant<|end_header_id|>1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. 2. Exercise regularly to keep your body active and strong. 3. Get enough sleep and maintain a consistent sleep schedule.<|eot_id|><|end_of_text|>']
['<|begin_of_text|><|start_header_id|>user<|end_header_id|>Give three tips for staying healthy.<|eot_id|><|start_header_id|>assistant<|end_header_id|>1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. 2. Exercise regularly to keep your body active and strong. 3. Get enough sleep and maintain a consistent sleep schedule.<|eot_id|><|end_of_text|>', '<|begin_of_text|><|start_header_id|>user<|end_header_id|>What are the three primary colors?<|eot_id|><|start_header_id|>assistant<|end_header_id|>The three primary colors are red, blue, and yellow.<|eot_id|><|end_of_text|>']


In [7]:
# encode formatted text
fukk_token_list = []

for i in range(len(formatted_text)):
    tokenizer.add_special_tokens({'pad_token': '<|end_of_text|>'})
    tokens = tokenizer(formatted_text[i], truncation=True, padding="max_length", max_length=dataset_max_length, return_tensors="pt")
    print(tokens)
    fukk_token_list.append(tokens)

{'input_ids': tensor([[128000, 128000, 128006,    882, 128007,  36227,   2380,  10631,    369,
          19994,   9498,     13, 128009, 128006,  78191, 128007,     16,   5253,
            266,    264,  24770,  10173,    323,   1304,   2771,    311,   2997,
          11510,    315,  26390,    323,  24822,     13,    220,     17,     13,
          33918,  15870,    311,   2567,    701,   2547,   4642,    323,   3831,
             13,    220,     18,     13,   2175,   3403,   6212,    323,  10519,
            264,  13263,   6212,   9899,     13, 128009, 128001, 128001, 128001,
         128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001,
         128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001,
         128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001,
         128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001,
         128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001,
         12800

In [29]:
input_ids = fukk_token_list[0]["input_ids"].squeeze()
print(input_ids.shape)
print(input_ids)

torch.Size([512])
tensor([128000, 128000, 128006,    882, 128007,  36227,   2380,  10631,    369,
         19994,   9498,     13, 128009, 128006,  78191, 128007,     16,   5253,
           266,    264,  24770,  10173,    323,   1304,   2771,    311,   2997,
         11510,    315,  26390,    323,  24822,     13,    220,     17,     13,
         33918,  15870,    311,   2567,    701,   2547,   4642,    323,   3831,
            13,    220,     18,     13,   2175,   3403,   6212,    323,  10519,
           264,  13263,   6212,   9899,     13, 128009, 128001, 128001, 128001,
        128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001,
        128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001,
        128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001,
        128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001,
        128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001,
        128001, 128001

In [10]:
attention_mask = fukk_token_list[0]["attention_mask"].squeeze()
print(attention_mask)

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [27]:
assistant_start = "<|start_header_id|>assistant<|end_header_id|>" # assistant部分までをプロンプトとして扱う

# if assistant_start in input_word_list[0]
if assistant_start in input_word_list[0]:
    prompt_end_pos = input_word_list[0].find(assistant_start) + len(assistant_start) # assistant開始位置までをプロンプトとして扱う
    prompt_text = input_word_list[0][:prompt_end_pos]
    print(prompt_end_pos)
    
    # tokenize prompt text
    prompt_tokens = tokenizer(prompt_text, truncation=True, padding=False, add_special_tokens=False)
    print(prompt_tokens)

    # mask labels
    labels = input_ids.clone() # ラベルを作成してプロンプト部分をマスク
    prompt_length = len(prompt_tokens["input_ids"])
    labels[:prompt_length] = -100 # パディングトークンをマスク
    print(labels)

    # test decode labels
    cleaning_attention_mask = labels[labels != -100]
    decoded_text = tokenizer.decode(cleaning_attention_mask)
    print(decoded_text)

    result = {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

<|end_header_id|>1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. 2. Exercise regularly to keep your body active and strong. 3. Get enough sleep and maintain a consistent sleep schedule.<|eot_id|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of